# USAID-Kenya-Sentiment-Analysis

### Extracting Data From X/Tweeter API (Tweepy)

In [1]:
!pip install --user tweepy



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import tweepy

# # Bearer token given

# # Initialize the Twitter client
# client = tweepy.Client(bearer_token=BEARER_TOKEN)

# # Query includes retweets, multiple keywords, and allows any language
# query = '(USAID OR "foreign aid" OR "funding cuts" OR "cut USAID funding" OR "cutting USAID" OR "halted USAID support" OR "cut funding" OR "usaid budget cuts" OR "NGO funding" OR "Africa aid" OR "USAID gets shut" OR "Trump cuts usaid" OR "USAID money" OR "aid withdrawal" OR "end of USAID" OR "Trump Cuts Aid To Africa" OR NGOs funding) (Kenya OR Nairobi)'

# # Fetch recent tweets (up to 100 at a time, max 7 days old)
# response = client.search_recent_tweets(query=query, max_results=100)

# # Print the tweets
# if response.data:
#     for tweet in response.data:
#         print(tweet.text)
# else:
#     print("No tweets found.")

## Extracting Data From Reddit API (PRAW)

In [3]:
!pip install praw




[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\commands\install.py", line 483, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\metadata\pkg_resources.py", line 192, in version
    return parse_version(s

In [4]:
import praw
import pandas as pd
from datetime import datetime

# Reddit API client
reddit = praw.Reddit(
    client_id='f6MuemGIIpuv8csr3CpuzQ',
    client_secret='rSPUbtIsCzfYWW3Lf9mnR330gX7FCA',
    user_agent='USAID-Ke-Prac'
)

# Keywords related to USAID, Kenya, Trump, and aid cuts
keywords = [
    "USAID", "foreign aid", "funding cuts", "aid budget", "Trump Kenya", 
    "USAID Kenya", "cut aid", "budget cuts", "NGO funding", "Africa aid",
    "USAID funding", "foreign aid", "Trump cuts", "USAID money", "aid withdrawal",
    "State Department cuts", "donors","end of USAID","Trump Cuts Aid To Africa","NGOs"
]

relevant_subreddits = ["Kenya","Nairobi"]

# the cutoff date
CUTOFF_DATE = datetime(2025, 1, 1)  

def search_reddit(subreddits, limit=100):
    posts = []
    
    for subreddit_name in subreddits:
        try:
            subreddit = reddit.subreddit(subreddit_name)
            for submission in subreddit.search("USAID OR Kenya OR aid", limit=limit, sort="new"):
                post_date = datetime.utcfromtimestamp(submission.created_utc)
                
                # Skip posts before the cutoff date
                if post_date < CUTOFF_DATE:
                    continue
                
                title_lower = submission.title.lower()
                selftext_lower = submission.selftext.lower()
                matched_keywords = [kw for kw in keywords if kw.lower() in title_lower or kw.lower() in selftext_lower]
                
                if matched_keywords:
                    posts.append({
                        'subreddit': subreddit_name,
                        'keyword': matched_keywords[0],
                        'title': submission.title,
                        'text': submission.selftext,
                        'date_posted': post_date.strftime('%Y-%m-%d %H:%M:%S'),
                        'score': submission.score,
                        'url': submission.url,
                        'permalink': f"https://reddit.com{submission.permalink}",
                        'upvotes': submission.score,
                        'comments': submission.num_comments
                    })
        except Exception as e:
            print(f"Skipped r/{subreddit_name}: {e}")
    
    return pd.DataFrame(posts)

df_reddit = search_reddit(relevant_subreddits, limit=300)
print(f"Found {len(df_reddit)} relevant posts after {CUTOFF_DATE}")
df_reddit.head(50)

Found 3 relevant posts after 2025-01-01 00:00:00


,subreddit,keyword,title,text,date_posted,score,url,permalink,upvotes,comments
0,Kenya,USAID,Want to self learn data analysis,Am looking for peeps willing to learn data ana...,2025-06-12 18:09:25,1,https://www.reddit.com/r/Kenya/comments/1l9tgy...,https://reddit.com/r/Kenya/comments/1l9tgyw/wa...,1,4
1,Nairobi,USAID,Theres always someone out there worse off than...,I was talking to my South Sudanese friend yest...,2025-05-12 23:28:50,81,https://www.reddit.com/r/nairobi/comments/1kl7...,https://reddit.com/r/nairobi/comments/1kl7bcm/...,81,53
2,Nairobi,aid withdrawal,What steps are you taking to keep your busines...,"Let's face it, since last year August; a lot o...",2025-04-14 09:54:50,8,https://www.reddit.com/r/nairobi/comments/1jyv...,https://reddit.com/r/nairobi/comments/1jyv2tj/...,8,4


In [20]:
import praw
import pandas as pd
from datetime import datetime

# Reddit API Setup
reddit = praw.Reddit(
    client_id='f6MuemGIIpuv8csr3CpuzQ',
    client_secret='rSPUbtIsCzfYWW3Lf9mnR330gX7FCA',
    user_agent='USAID-Kenya-Aid-Analysis'
)

# Kenya-Specific Keywords
keywords = [
    "USAID", "foreign aid", "funding cuts", "aid budget", "Trump Kenya", 
    "USAID Kenya", "cut aid", "budget cuts", "NGO funding", "Africa aid",
    "USAID funding", "foreign aid", "Trump cuts", "USAID money", "aid withdrawal",
    "State Department cuts", "donors", "end of USAID", "Trump Cuts Aid To Africa", "NGOs"
]

# Only Kenya-focused subreddits
relevant_subreddits = ["Kenya", "Nairobi"]  

# Cutoff date (March 2025)
CUTOFF_DATE = datetime(2025, 1, 1)  # Adjust to exact announcement date if needed

def search_reddit_kenya(subreddits, limit=100):
    posts = []
    
    for subreddit_name in subreddits:
        try:
            subreddit = reddit.subreddit(subreddit_name)
            # Search for "Kenya + aid" in subreddits
            for submission in subreddit.search("Kenya AND (USAID OR aid OR funding)", limit=limit, sort="new"):
                post_date = datetime.utcfromtimestamp(submission.created_utc)
                
                # Skip posts before the cutoff date
                if post_date < CUTOFF_DATE:
                    continue
                
                # Check for Kenya-specific keywords in title/text
                content = f"{submission.title} {submission.selftext}".lower()
                matched_keywords = [kw for kw in keywords if kw.lower() in content]
                
                if matched_keywords:
                    posts.append({
                        'subreddit': subreddit_name,
                        'keyword': ", ".join(matched_keywords),
                        'title': submission.title,
                        'text': submission.selftext,
                        'date_posted': post_date.strftime('%Y-%m-%d'),
                        'upvotes': submission.score,
                        'comments': submission.num_comments,
                        'url': submission.url,  # Direct URL to linked content
                        'permalink': f"https://reddit.com{submission.permalink}"  # Link to Reddit discussion
                    })
        except Exception as e:
            print(f"Error in r/{subreddit_name}: {e}")
    
    return pd.DataFrame(posts)

# Run search
df_kenya = search_reddit_kenya(relevant_subreddits, limit=200)
print(f"Found {len(df_kenya)} Kenya-focused posts after {CUTOFF_DATE.date()}")
df_kenya.head(70)

Found 31 Kenya-focused posts after 2025-01-01


,subreddit,keyword,title,text,date_posted,upvotes,comments,url,permalink
0,Kenya,donors,Dear Gentle Readers,Good morning my gentle readers\n\nToday I have...,2025-05-30,51,73,https://www.reddit.com/r/Kenya/comments/1kyxeb...,https://reddit.com/r/Kenya/comments/1kyxebr/de...
1,Kenya,USAID,USAID Repercussions + Economy,My neighbour’s wife was a very big shot in USA...,2025-05-14,13,32,https://www.reddit.com/r/Kenya/comments/1kmhn8...,https://reddit.com/r/Kenya/comments/1kmhn87/us...
2,Kenya,Trump cuts,Bill Gates 'horrified' by Trump cuts to US aid...,,2025-05-08,1,1,https://www.semafor.com/article/05/08/2025/bil...,https://reddit.com/r/Kenya/comments/1khpakg/bi...
3,Kenya,"foreign aid, foreign aid",Foreign aid/Philanthropy,I see the aid or philanthropic activities that...,2025-04-27,1,0,https://www.reddit.com/r/Kenya/comments/1k91v5...,https://reddit.com/r/Kenya/comments/1k91v58/fo...
4,Kenya,USAID,"USAID left a month ago, do we have ARVs in Kenya?",Someone on a different group (different websit...,2025-04-15,3,5,https://www.reddit.com/r/Kenya/comments/1jzrn2...,https://reddit.com/r/Kenya/comments/1jzrn2s/us...
5,Kenya,USAID,Classism in r/Kenya and r/nairobi,The classism I'm seeing in both subs is a good...,2025-04-07,166,95,https://www.reddit.com/r/Kenya/comments/1jtcvb...,https://reddit.com/r/Kenya/comments/1jtcvbx/cl...
6,Kenya,USAID,Economy,For the experts in matters economy and finance...,2025-04-06,1,15,https://www.reddit.com/r/Kenya/comments/1jsyty...,https://reddit.com/r/Kenya/comments/1jsytyp/ec...
7,Kenya,USAID,EX-USAID people!! Let's talk,Are you still in contact with the organisation...,2025-04-05,2,2,https://www.reddit.com/r/Kenya/comments/1jsb14...,https://reddit.com/r/Kenya/comments/1jsb149/ex...
8,Kenya,"foreign aid, foreign aid",Kibaki ALSO failed us,\nThere is a tendency to over-exaggerate the p...,2025-04-01,120,124,https://www.reddit.com/r/Kenya/comments/1jojl2...,https://reddit.com/r/Kenya/comments/1jojl2f/ki...
9,Kenya,"foreign aid, foreign aid",Is Data the missing link for Kenya's entrepren...,Africa’s economic future is being redefined by...,2025-03-30,2,5,https://www.reddit.com/r/Kenya/comments/1jnfss...,https://reddit.com/r/Kenya/comments/1jnfssf/is...


In [21]:
df_kenya.shape

(31, 9)

In [5]:
import praw
import pandas as pd
from datetime import datetime

# Reddit API Setup
reddit = praw.Reddit(
    client_id='f6MuemGIIpuv8csr3CpuzQ',
    client_secret='rSPUbtIsCzfYWW3Lf9mnR330gX7FCA',
    user_agent='USAID-Kenya-Aid-Analysis'
)

# Expanded Kenya-Specific Keywords
keywords = [
   "USAID", "foreign aid", "funding cuts", "aid budget", "Trump Kenya", 
    "USAID Kenya", "cut aid", "budget cuts", "NGO funding", "Africa aid",
    "USAID funding", "foreign aid", "Trump cuts", "USAID money", "aid withdrawal",
    "State Department cuts", "donors", "end of USAID Kenya", "Trump Cuts Aid To Africa", "NGOs","USAID exit Kenya","USAID Repercussions"
]
# Expanded subreddits list
relevant_subreddits = [
    "Kenya", "Nairobi", "Africa", "worldnews", "geopolitics",
    "InternationalDev", "humanitarian", "globalhealth", "development",'politics','AskConservatives','news'
]

# Adjusted cutoff date (since we're getting 0 results for 2025)
CUTOFF_DATE = datetime(2025, 1, 1)  # Changed to 2023 to get more results

def search_reddit_kenya(subreddits, limit=100):
    posts = []
    
    for subreddit_name in subreddits:
        try:
            print(f"Searching r/{subreddit_name}...")
            subreddit = reddit.subreddit(subreddit_name)
            
            # Try different search queries
            search_queries = [
                "Kenya AND (USAID OR aid OR funding)",
                "Kenya AND (development OR assistance)",
                "USAID Kenya",
                "foreign aid Kenya"
            ]
            
            for query in search_queries:
                try:
                    for submission in subreddit.search(query, limit=limit, sort="new"):
                        post_date = datetime.utcfromtimestamp(submission.created_utc)
                        
                        if post_date < CUTOFF_DATE:
                            continue
                            
                        content = f"{submission.title} {submission.selftext}".lower()
                        matched_keywords = [kw for kw in keywords if kw.lower() in content]
                        
                        if matched_keywords:
                            posts.append({
                                'subreddit': subreddit_name,
                                'keyword': ", ".join(matched_keywords),
                                'title': submission.title,
                                'text': submission.selftext,
                                'date_posted': post_date.strftime('%Y-%m-%d'),
                                'upvotes': submission.score,
                                'comments': submission.num_comments,
                                'url': submission.url,
                                'permalink': f"https://reddit.com{submission.permalink}"
                            })
                except Exception as e:
                    print(f"Error with query '{query}' in r/{subreddit_name}: {e}")
                    
        except Exception as e:
            print(f"Error accessing r/{subreddit_name}: {e}")
    
    return pd.DataFrame(posts)

# Run search
print("Starting Reddit search...")
df_kenya = search_reddit_kenya(relevant_subreddits, limit=200)

# Print summary
print("\nSearch completed!")
print(f"Found {len(df_kenya)} Kenya-focused posts after {CUTOFF_DATE.date()}")

# Save to CSV with timestamp
if not df_kenya.empty:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"ceciliareddit_kenya_aid_posts_{timestamp}.csv"
    df_kenya.to_csv(output_filename, index=False, encoding='utf-8')
    print(f"Results saved to {output_filename}")
    

df_kenya.head()

Starting Reddit search...
Searching r/Kenya...
Searching r/Nairobi...
Searching r/Africa...
Searching r/worldnews...
Searching r/geopolitics...
Searching r/InternationalDev...
Searching r/humanitarian...
Searching r/globalhealth...
Searching r/development...
Searching r/politics...
Searching r/AskConservatives...
Searching r/news...

Search completed!
Found 254 Kenya-focused posts after 2025-01-01


,subreddit,keyword,title,text,date_posted,upvotes,comments,url,permalink
0,Kenya,donors,Dear Gentle Readers,Good morning my gentle readers\n\nToday I have...,2025-05-30,53,73,https://www.reddit.com/r/Kenya/comments/1kyxeb...,https://reddit.com/r/Kenya/comments/1kyxebr/de...
1,Kenya,"USAID, USAID Repercussions",USAID Repercussions + Economy,My neighbour’s wife was a very big shot in USA...,2025-05-14,13,32,https://www.reddit.com/r/Kenya/comments/1kmhn8...,https://reddit.com/r/Kenya/comments/1kmhn87/us...
2,Kenya,Trump cuts,Bill Gates 'horrified' by Trump cuts to US aid...,,2025-05-08,1,1,https://www.semafor.com/article/05/08/2025/bil...,https://reddit.com/r/Kenya/comments/1khpakg/bi...
3,Kenya,"foreign aid, foreign aid",Foreign aid/Philanthropy,I see the aid or philanthropic activities that...,2025-04-27,1,0,https://www.reddit.com/r/Kenya/comments/1k91v5...,https://reddit.com/r/Kenya/comments/1k91v58/fo...
4,Kenya,USAID,"USAID left a month ago, do we have ARVs in Kenya?",Someone on a different group (different websit...,2025-04-15,3,5,https://www.reddit.com/r/Kenya/comments/1jzrn2...,https://reddit.com/r/Kenya/comments/1jzrn2s/us...


In [6]:
df_kenya.shape

(254, 9)

In [8]:
df_kenya.sample(20)

,subreddit,keyword,title,text,date_posted,upvotes,comments,url,permalink
1,Kenya,"USAID, USAID Repercussions",USAID Repercussions + Economy,My neighbour’s wife was a very big shot in USA...,2025-05-14,13,32,https://www.reddit.com/r/Kenya/comments/1kmhn8...,https://reddit.com/r/Kenya/comments/1kmhn87/us...
160,politics,"foreign aid, foreign aid",Judge Won’t Force Trump to Restore Foreign Aid...,,2025-03-11,113,13,https://www.reuters.com/world/us/judge-will-no...,https://reddit.com/r/politics/comments/1j8hm3o...
29,Kenya,"USAID, USAID money",Why western powers back Israel no matter what ...,"I don't care what good book you read, but it's...",2025-03-25,13,20,https://www.reddit.com/r/Kenya/comments/1jjehw...,https://reddit.com/r/Kenya/comments/1jjehwq/wh...
84,InternationalDev,USAID,Are any of you in Peru or do work with the Per...,I’m a recreational travel blogger and I’m in t...,2025-06-04,28,7,https://i.redd.it/sa7ytqwtuy4f1.jpeg,https://reddit.com/r/InternationalDev/comments...
128,humanitarian,"USAID, foreign aid, foreign aid",Foreign Aid Freeze on USAID,The Trump Administration has implemented a fre...,2025-01-29,13,7,https://www.reddit.com/r/humanitarian/comments...,https://reddit.com/r/humanitarian/comments/1ic...
115,InternationalDev,"foreign aid, foreign aid","In 2017, 120 former US military leaders sent a...",This is the message we need to get out to ever...,2025-02-01,184,0,http://www.usglc.org/downloads/2017/02/FY18_In...,https://reddit.com/r/InternationalDev/comments...
200,politics,"foreign aid, foreign aid",Majority of Americans believe foreign aid ‘was...,,2025-02-17,0,43,https://www.ft.com/content/c92c76d1-02cc-45fb-...,https://reddit.com/r/politics/comments/1irq4jo...
176,politics,"foreign aid, foreign aid",Supreme Court rejects Trump’s request to keep ...,,2025-03-05,57,13,https://edition.cnn.com/2025/03/05/politics/su...,https://reddit.com/r/politics/comments/1j44jrs...
180,politics,USAID,Supreme Court Rejects Trump on USAID Foreign-A...,,2025-03-05,626,60,https://www.bloomberg.com/news/articles/2025-0...,https://reddit.com/r/politics/comments/1j43z3n...
119,InternationalDev,"foreign aid, foreign aid",Impact of Judge blocking Trump's Spending Freeze,What do we think the albeit temporary good new...,2025-01-28,14,5,https://www.reddit.com/r/InternationalDev/comm...,https://reddit.com/r/InternationalDev/comments...


In [17]:
import praw
import pandas as pd
from datetime import datetime

# Reddit API Setup
reddit = praw.Reddit(
    client_id='f6MuemGIIpuv8csr3CpuzQ',
    client_secret='rSPUbtIsCzfYWW3Lf9mnR330gX7FCA',
    user_agent='USAID-Kenya-Aid-Analysis'
)

# Expanded Kenya-Specific Keywords
keywords = [
   "USAID", "foreign aid", "funding cuts", "aid budget", "Trump Kenya", 
    "USAID Kenya", "cut aid", "budget cuts", "NGO funding", "Africa aid",
    "USAID funding", "foreign aid", "Trump cuts", "USAID money", "aid withdrawal",
    "State Department cuts", "donors", "end of USAID Kenya", "Trump Cuts Aid To Africa", "NGOs","USAID exit Kenya","USAID Repercussions"
]
# Expanded subreddits list
relevant_subreddits = [
    "Kenya", "Nairobi", "Africa", "worldnews", "geopolitics",
    "InternationalDev", "humanitarian", "globalhealth", "development",'politics','AskConservatives','news'
]

# Adjusted cutoff date (since we're getting 0 results for 2025)
CUTOFF_DATE = datetime(2025, 1, 1)  # Changed to 2023 to get more results

def search_reddit_kenya(subreddits, limit=100):
    posts = []
    
    for subreddit_name in subreddits:
        try:
            print(f"Searching r/{subreddit_name}...")
            subreddit = reddit.subreddit(subreddit_name)
            
            # Try different search queries
            search_queries = [
                "Kenya AND (USAID OR aid OR funding)",
                "Kenya AND (development OR assistance)",
                "USAID Kenya",
                "foreign aid Kenya"
            ]
            
            for query in search_queries:
                try:
                    for submission in subreddit.search(query, limit=limit, sort="new"):
                        post_date = datetime.utcfromtimestamp(submission.created_utc)
                        
                        if post_date < CUTOFF_DATE:
                            continue
                            
                        content = f"{submission.title} {submission.selftext}".lower()
                        matched_keywords = [kw for kw in keywords if kw.lower() in content]
                        
                        if matched_keywords:
                            posts.append({
                                'subreddit': subreddit_name,
                                'keyword': ", ".join(matched_keywords),
                                'title': submission.title,
                                'text': submission.selftext,
                                'date_posted': post_date.strftime('%Y-%m-%d'),
                                'upvotes': submission.score,
                                'comments': submission.num_comments,
                                'url': submission.url,
                                'permalink': f"https://reddit.com{submission.permalink}"
                            })
                except Exception as e:
                    print(f"Error with query '{query}' in r/{subreddit_name}: {e}")
                    
        except Exception as e:
            print(f"Error accessing r/{subreddit_name}: {e}")
    
    return pd.DataFrame(posts)

# Run search
print("Starting Reddit search...")
df_kenya2 = search_reddit_kenya(relevant_subreddits, limit=200)

# Print summary
print("\nSearch completed!")
print(f"Found {len(df_kenya)} Kenya-focused posts after {CUTOFF_DATE.date()}")

# # Save to CSV with timestamp
# if not df_kenya.empty:
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     output_filename = f"reddit_kenya_aid_posts_{timestamp}.csv"
#     df_kenya.to_csv(output_filename, index=False, encoding='utf-8')
#     print(f"Results saved to {output_filename}")
    
#     # Display sample
# print("\nSample results:")
# print(df_kenya[['subreddit', 'title', 'date_posted']].head())
#     else:
# print("No posts found. Consider adjusting your search parameters.")
df_kenya2.head()

Starting Reddit search...
Searching r/Kenya...
Searching r/Nairobi...
Searching r/Africa...
Searching r/worldnews...
Searching r/geopolitics...
Searching r/InternationalDev...
Searching r/humanitarian...
Searching r/globalhealth...
Searching r/development...
Searching r/politics...
Searching r/AskConservatives...
Searching r/news...

Search completed!
Found 254 Kenya-focused posts after 2025-01-01


,subreddit,keyword,title,text,date_posted,upvotes,comments,url,permalink
0,Kenya,donors,Dear Gentle Readers,Good morning my gentle readers\n\nToday I have...,2025-05-30,53,73,https://www.reddit.com/r/Kenya/comments/1kyxeb...,https://reddit.com/r/Kenya/comments/1kyxebr/de...
1,Kenya,"USAID, USAID Repercussions",USAID Repercussions + Economy,My neighbour’s wife was a very big shot in USA...,2025-05-14,14,32,https://www.reddit.com/r/Kenya/comments/1kmhn8...,https://reddit.com/r/Kenya/comments/1kmhn87/us...
2,Kenya,Trump cuts,Bill Gates 'horrified' by Trump cuts to US aid...,,2025-05-08,1,1,https://www.semafor.com/article/05/08/2025/bil...,https://reddit.com/r/Kenya/comments/1khpakg/bi...
3,Kenya,"foreign aid, foreign aid",Foreign aid/Philanthropy,I see the aid or philanthropic activities that...,2025-04-27,1,0,https://www.reddit.com/r/Kenya/comments/1k91v5...,https://reddit.com/r/Kenya/comments/1k91v58/fo...
4,Kenya,USAID,"USAID left a month ago, do we have ARVs in Kenya?",Someone on a different group (different websit...,2025-04-15,3,5,https://www.reddit.com/r/Kenya/comments/1jzrn2...,https://reddit.com/r/Kenya/comments/1jzrn2s/us...


In [19]:
df_kenya2.sample(20)


,subreddit,keyword,title,text,date_posted,upvotes,comments,url,permalink
113,InternationalDev,"foreign aid, foreign aid, donors","After the US shock, what are your resource mob...","Hello everyone, long-time lurker here and lear...",2025-02-03,13,0,https://www.reddit.com/r/InternationalDev/comm...,https://reddit.com/r/InternationalDev/comments...
124,InternationalDev,"foreign aid, foreign aid",Trump administration just suspended all new fo...,,2025-01-24,239,50,https://bsky.app/profile/kenklippenstein.bsky....,https://reddit.com/r/InternationalDev/comments...
139,politics,"foreign aid, foreign aid",House will vote on Trump's request to cut fund...,,2025-06-12,31,6,https://www.washingtonpost.com/politics/2025/0...,https://reddit.com/r/politics/comments/1l9pqcb...
129,humanitarian,"foreign aid, foreign aid",Predictions for the short term,Trump's executive order on foreign aid could j...,2025-01-27,12,4,https://www.reddit.com/r/humanitarian/comments...,https://reddit.com/r/humanitarian/comments/1ib...
110,InternationalDev,"foreign aid, foreign aid","Public Citizen sues the Trump Administration, ...",Finally.,2025-02-11,68,0,https://www.citizen.org/news/nonprofits-sue-tr...,https://reddit.com/r/InternationalDev/comments...
181,politics,"foreign aid, foreign aid",DOGE's Foreign Aid Cuts Have Sparked 'Total Ch...,,2025-03-04,169,16,https://www.wired.com/story/rights-con-taipei-...,https://reddit.com/r/politics/comments/1j3i8s6...
118,InternationalDev,"foreign aid, foreign aid",The stop work orders are blatantly illegal. Wh...,It seems like no one is suing the Trump admini...,2025-01-29,49,1,https://www.reddit.com/r/InternationalDev/comm...,https://reddit.com/r/InternationalDev/comments...
66,worldnews,"foreign aid, foreign aid","As Trump guts foreign aid, China and others le...",,2025-03-31,3191,222,https://www.nbcnews.com/news/world/myanmar-ear...,https://reddit.com/r/worldnews/comments/1jo2eb...
232,politics,"foreign aid, foreign aid","US orders immediate pause to foreign aid, leak...",,2025-01-26,114,67,https://www.bbc.com/news/articles/ce9nx5k7lv0o...,https://reddit.com/r/politics/comments/1ia2ze9...
138,politics,"foreign aid, foreign aid",House Republicans vote to claw back $9.4 billi...,,2025-06-12,79,15,https://www.nbcnews.com/politics/congress/hous...,https://reddit.com/r/politics/comments/1l9weme...


In [10]:
# # Save to CSV
# output_filename = "reddit_kenya_aid_posts.csv"
# df_kenya.to_csv(output_filename, index=False, encoding='utf-8')
# print(f"Data saved to {output_filename}")

## Extracting Data From News API (NewsAPI.org)

In [7]:
!pip install requests


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\commands\install.py", line 483, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\pip\_internal\metadata\pkg_resources.py", line 192, in version
    return parse_version(se

In [8]:
import requests
import pandas as pd
import time

# My NewsAPI key
API_KEY = 'e11a1726c9c1470ea9abc0eead154d6c'

# Base URL
BASE_URL = 'https://newsapi.org/v2/everything'

# Keywords to search
keywords = [
    "USAID", "foreign aid", "funding cuts", "aid budget", "Trump Kenya",
    "USAID Kenya", "cut aid", "budget cuts", "NGO funding", "Africa aid",
    "USAID funding", "foreign aid", "Trump cuts", "USAID money", "aid withdrawal",
    "State Department cuts", "donors", "end of USAID", "Trump Cuts Aid To Africa", "NGOs"
]


all_articles = []

# Loop through each keyword
for keyword in keywords:
    print(f"Fetching articles for: {keyword}")
    
    params = {
        'q': keyword,
        'apiKey': API_KEY,
        'language': 'en',
        'sortBy': 'relevancy',
        'pageSize': 100,
    }

    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        for article in articles:
            all_articles.append({
                'keyword': keyword,
                'source': article['source']['name'],
                'title': article['title'],
                'description': article['description'],
                'url': article['url'],
                'publishedAt': article['publishedAt']
            })
    else:
        print(f"Error for keyword '{keyword}':", response.status_code, response.text)
    
    # To avoid rate limiting
    time.sleep(1.5)

# Convert to DataFrame
df = pd.DataFrame(all_articles)
df.sample (50)

Fetching articles for: USAID
Fetching articles for: foreign aid
Fetching articles for: funding cuts
Fetching articles for: aid budget
Fetching articles for: Trump Kenya
Fetching articles for: USAID Kenya
Fetching articles for: cut aid
Fetching articles for: budget cuts
Fetching articles for: NGO funding
Fetching articles for: Africa aid
Fetching articles for: USAID funding
Fetching articles for: foreign aid
Fetching articles for: Trump cuts
Fetching articles for: USAID money
Fetching articles for: aid withdrawal
Fetching articles for: State Department cuts
Fetching articles for: donors
Fetching articles for: end of USAID
Fetching articles for: Trump Cuts Aid To Africa
Fetching articles for: NGOs


,keyword,source,title,description,url,publishedAt
1157,USAID money,Time,Why NGOs in the Philippines Keep Being Charged...,And how the Trump Administration could be taki...,https://time.com/7285803/philippines-ngo-red-t...,2025-05-16T07:30:00Z
560,cut aid,Forbes,House Moves To Claw Back Money From Public Bro...,The House is seeking to rescind $9.4 billion i...,https://www.forbes.com/sites/kellyphillipserb/...,2025-06-12T14:06:37Z
28,USAID,Naturalnews.com,"Rubio terminates all USAID overseas positions,...",Secretary Rubio orders termination of all USAI...,https://www.naturalnews.com/2025-06-11-rubio-t...,2025-06-11T06:00:00Z
530,cut aid,Al Jazeera English,Congolese refugees in Burundi face starvation ...,Shortages and security worsen in Burundi's ove...,https://www.aljazeera.com/gallery/2025/5/26/co...,2025-05-26T11:41:40Z
970,foreign aid,BBC News,UK sanctions 'godmother' of Israel's settler m...,"The foreign secretary said the move ""demonstra...",https://www.bbc.com/news/articles/c7066e7z9k4o,2025-05-20T16:00:36Z
130,foreign aid,BBC News,'It's not fair': Other refugees in limbo as US...,Refugees from other nations ask why the White ...,https://www.bbc.com/news/articles/czellw10ejdo,2025-05-24T23:10:45Z
134,foreign aid,Vox,Two terrible truths about the antisemitic murd...,"On Wednesday evening, the American Jewish Comm...",https://www.vox.com/politics/414137/dc-shootin...,2025-05-22T22:25:30Z
1711,NGOs,Ra.co,'Not a single euro to KKR': Sónar further addr...,The Barcelona festival also pledged to make a ...,https://ra.co/news/82876,2025-05-29T13:15:00Z
131,foreign aid,Theregister.com,"Russia's Fancy Bear swipes a paw at logistics,...",13 govt agencies sound the alarm\nRussian cybe...,https://www.theregister.com/2025/05/21/russias...,2025-05-21T19:16:29Z
263,funding cuts,Al Jazeera English,EU to sustain Radio Free Europe with emergency...,Outlet is one of several media services whose ...,https://www.aljazeera.com/news/2025/5/20/eu-to...,2025-05-20T19:58:34Z


In [10]:
df.shape

(1780, 6)